In [1]:
from os.path import dirname, basename

import pandas as pd

from inpainting.load import load_dataframe
from inpainting.utils import get_paths


In [2]:
dataset = 'demo'
results_dirs = get_paths(f'../results/ii/{dataset}/*')

times_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_df = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(times_df, results_df, on=['model', 'sequence_name', 'frame_id'])
df

,frame_id,inference_time,model,sequence_name,mean_squared_error,peak_signal_noise_ratio,structural_similarity,object_sequence_name,object_id
0,0,62.915039,ii_deepfillv1,breakdance_paragliding_002,86.109901,28.780273,0.978142,paragliding,2
1,1,61.121952,ii_deepfillv1,breakdance_paragliding_002,58.115443,30.487888,0.980445,paragliding,2
2,2,59.795296,ii_deepfillv1,breakdance_paragliding_002,48.550227,31.268891,0.981613,paragliding,2
3,3,58.852512,ii_deepfillv1,breakdance_paragliding_002,48.286463,31.292550,0.982666,paragliding,2
4,4,55.847679,ii_deepfillv1,breakdance_paragliding_002,63.719376,30.088088,0.980639,paragliding,2
...,...,...,...,...,...,...,...,...,...
4902,42,56.940353,ii_regionfill,tennis_soccerball_001,2.252795,44.603587,0.995202,soccerball,1
4903,43,51.636192,ii_regionfill,tennis_soccerball_001,2.061495,44.988980,0.995386,soccerball,1
4904,44,56.927937,ii_regionfill,tennis_soccerball_001,2.263781,44.582459,0.995440,soccerball,1
4905,45,53.320255,ii_regionfill,tennis_soccerball_001,2.233800,44.640360,0.995453,soccerball,1


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['inference_time'] = df['inference_time']
metrics['mean_squared_error'] = df['mean_squared_error']
metrics['peak_signal_noise_ratio'] = df['peak_signal_noise_ratio']
metrics['structural_similarity'] = df['structural_similarity']
metrics.groupby('model').mean()

,inference_time,mean_squared_error,peak_signal_noise_ratio,structural_similarity
model,,,,
ii_deepfillv1,50.953780,72.384135,32.050414,0.966203
ii_deepfillv1_flownet2,332.814639,36.206546,37.708112,0.983929
ii_deepfillv1_pwcnet,265.981093,37.352511,37.493410,0.984077
ii_deepfillv2,30.193220,57.447578,32.169737,0.967450
ii_kernelfill,237.754556,44.727686,34.166850,0.972217
ii_pconvunet,22.788819,179.694561,27.049354,0.955417
ii_regionfill,94.253227,40.960994,34.484699,0.972674
